In [1]:
# ACC 측정
y_pred = []

# Test 기간 설정 : 2019년 2월 1일 ~ 2020년 1월 31일
import pandas as pd
data = pd.read_csv('../data/er_data.csv')
data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']
data = data.set_index('DATE')
test_days = list(data['2019-01-01':'2019-12-31'].index.values)

for test_day in test_days :
    
    # 데이터 불러오기
    import pandas as pd
    data = pd.read_csv('../data/er_data.csv')

    # Dataset Columns
    data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']

    # Dataset Index
    data = data.set_index('DATE')
    
    # 데이터 shift 및 라벨링
    data['USD_1'] = data['USD'].shift(-1)
    data['Target'] = data['USD_1'] - data['USD'] * 1.001
    data['Target'] = data['Target'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
    data = data.dropna()
    
    # Feature Selecting
    data = data[['CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','Target']]
    
    # 데이터 갱신
    data = data[:test_day]
    
    # Test 데이터 기준 최근 200일만 학습에 사용. 하루 예측하고, 모델 갱신
    data = data[-201:]
    train = data[:-1]
    test = data[-1:]
    
    # Target, Input Split
    train_input = train[['CAD',"CHF","EUR",'GBP','JPY(100)','BITCOIN','TETHER']]
    train_target = train['Target']

    test_input = test[['CAD',"CHF","EUR",'GBP','JPY(100)','BITCOIN','TETHER']]
    test_target = test['Target']

    # Transform to Numpy Array
    import numpy as np
    train = np.array(train)
    test = np.array(test)
    
    # Data Normalization
    mean = train_input.mean(axis=0)
    train_input -= mean
    std = train_input.std(axis=0)
    train_input /= std
    test_input -= mean
    test_input /= std
    
    # XGBoost Build
    from xgboost import XGBClassifier
    model = XGBClassifier(booster='gbtree', 
                          colsample_bylevel=0.9, 
                          colsample_bytree=0.8, 
                          gamma=0, 
                          max_depth=5, 
                          min_child_weight=3, 
                          n_estimators=10, 
                          nthread=-1, 
                          objective='binary:logistic', 
                          random_state=0, 
                          silent= True,
                          learning_rate = 0.1)

    # Model Fitting
    model.fit(train_input,train_target, eval_set=[(train_input, train_target)], early_stopping_rounds=50,verbose=0)
    
    # Acc Check
    from sklearn.metrics import accuracy_score
    y_pred.append(model.predict(test_input)[0])
    
# y_true
data = pd.read_csv('../data/er_data.csv')
data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']
data = data.set_index('DATE')
data['USD_1'] = data['USD'].shift(-1)
data['Target'] = data['USD_1'] - data['USD'] * 1.001
data['Target'] = data['Target'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
data = data.dropna()
data = data[['CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','Target']]
y_true = data['2019-01-02':'2019-12-31']['Target']

# ACC
print('Online Learning ACC = {:.2f}%'.format(accuracy_score(y_true,y_pred)*100))

Online Learning ACC = 58.94%


In [2]:
# ACC 측정
y_pred = []

# Test 기간 설정 : 2019년 2월 1일 ~ 2020년 1월 31일
import pandas as pd
data = pd.read_csv('../data/er_data.csv')
data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']
data = data.set_index('DATE')
test_days = list(data['2019-01-01':'2019-12-31'].index.values)

for test_day in test_days :
    
    # 데이터 불러오기
    import pandas as pd
    data = pd.read_csv('../data/er_data.csv')

    # Dataset Columns
    data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']

    # Dataset Index
    data = data.set_index('DATE')
    
    # 데이터 shift 및 라벨링
    data['USD_1'] = data['USD'].shift(-1)
    data['Target'] = data['USD_1'] - data['USD'] * 1.001
    data['Target'] = data['Target'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
    data = data.dropna()
    
    # Feature Selecting
    data = data[['CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','Target']]
    
    # 데이터 갱신
    data = data[:test_day]
    
    # Test 데이터 기준 최근 200일만 학습에 사용. 하루 예측하고, 모델 갱신
    data = data[-201:]
    train = data[:-1]
    test = data[-1:]
    
    # Target, Input Split
    train_input = train[['CAD',"CHF","EUR",'GBP','JPY(100)','BITCOIN','TETHER']]
    train_target = train['Target']

    test_input = test[['CAD',"CHF","EUR",'GBP','JPY(100)','BITCOIN','TETHER']]
    test_target = test['Target']

    # Transform to Numpy Array
    import numpy as np
    train = np.array(train)
    test = np.array(test)
    
    # Data Normalization
    mean = train_input.mean(axis=0)
    train_input -= mean
    std = train_input.std(axis=0)
    train_input /= std
    test_input -= mean
    test_input /= std
    
    # GridSearch
    from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
    params = {
            'min_child_weight': [1, 5, 10],
            'gamma': [0.5, 1, 1.5, 2, 5],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'max_depth': [3, 4, 5]
            }
    param_comb = 5
    # Model Build
    xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)
    random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, n_jobs=-1,\
                                       verbose=1,random_state=1001)

    random_search.fit(train_input,train_target)
    
    # Acc Check
    from sklearn.metrics import accuracy_score
    y_pred.append(model.predict(test_input)[0])
    
# y_true
data = pd.read_csv('../data/er_data.csv')
data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']
data = data.set_index('DATE')
data['USD_1'] = data['USD'].shift(-1)
data['Target'] = data['USD_1'] - data['USD'] * 1.001
data['Target'] = data['Target'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
data = data.dropna()
data = data[['CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','Target']]
y_true = data['2019-01-02':'2019-12-31']['Target']

# ACC
print('Online Learning ACC = {:.2f}%'.format(accuracy_score(y_true,y_pred)*100))

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    4.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


...

[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    2.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    2.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Online Learning ACC = 59.76%


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    2.4s finished


In [4]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best hyperparameters:')
print(random_search.best_params_)


 All results:
{'mean_fit_time': array([2.29464641, 2.35922966, 2.36588182, 2.36255903, 2.35190816]), 'std_fit_time': array([0.0469727 , 0.00280835, 0.00050351, 0.00157204, 0.00446766]), 'mean_score_time': array([0.00553331, 0.00486135, 0.00421209, 0.00320787, 0.00287981]), 'std_score_time': array([0.00102095, 0.00027421, 0.00064746, 0.00061771, 0.00012561]), 'param_subsample': masked_array(data=[1.0, 0.6, 0.8, 1.0, 0.8],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 1, 5, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 5, 5, 5, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[5, 1.5, 1, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dt

### Best Hyperparameter

In [6]:
# ACC 측정
y_pred = []

# Test 기간 설정 : 2019년 2월 1일 ~ 2020년 1월 31일
import pandas as pd
data = pd.read_csv('../data/er_data.csv')
data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']
data = data.set_index('DATE')
test_days = list(data['2019-01-01':'2019-12-31'].index.values)

for test_day in test_days :
    
    # 데이터 불러오기
    import pandas as pd
    data = pd.read_csv('../data/er_data.csv')

    # Dataset Columns
    data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']

    # Dataset Index
    data = data.set_index('DATE')
    
    # 데이터 shift 및 라벨링
    data['USD_1'] = data['USD'].shift(-1)
    data['Target'] = data['USD_1'] - data['USD'] * 1.001
    data['Target'] = data['Target'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
    data = data.dropna()
    
    # Feature Selecting
    data = data[['CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','Target']]
    
    # 데이터 갱신
    data = data[:test_day]
    
    # Test 데이터 기준 최근 200일만 학습에 사용. 하루 예측하고, 모델 갱신
    data = data[-201:]
    train = data[:-1]
    test = data[-1:]
    
    # Target, Input Split
    train_input = train[['CAD',"CHF","EUR",'GBP','JPY(100)','BITCOIN','TETHER']]
    train_target = train['Target']

    test_input = test[['CAD',"CHF","EUR",'GBP','JPY(100)','BITCOIN','TETHER']]
    test_target = test['Target']

    # Transform to Numpy Array
    import numpy as np
    train = np.array(train)
    test = np.array(test)
    
    # Data Normalization
    mean = train_input.mean(axis=0)
    train_input -= mean
    std = train_input.std(axis=0)
    train_input /= std
    test_input -= mean
    test_input /= std
    
    # XGBoost Build
    from xgboost import XGBClassifier
    model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=1,
              learning_rate=0.02, max_delta_step=0, max_depth=5,
              min_child_weight=5, missing=None, n_estimators=600, n_jobs=1,
              nthread=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.8, verbosity=1)

    # Model Fitting
    model.fit(train_input,train_target, eval_set=[(train_input, train_target)], early_stopping_rounds=50,verbose=0)
    
    # Acc Check
    from sklearn.metrics import accuracy_score
    y_pred.append(model.predict(test_input)[0])
    
# y_true
data = pd.read_csv('../data/er_data.csv')
data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']
data = data.set_index('DATE')
data['USD_1'] = data['USD'].shift(-1)
data['Target'] = data['USD_1'] - data['USD'] * 1.001
data['Target'] = data['Target'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
data = data.dropna()
data = data[['CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','Target']]
y_true = data['2019-01-02':'2019-12-31']['Target']

# ACC
print('Online Learning ACC = {:.2f}%'.format(accuracy_score(y_true,y_pred)*100))

Online Learning ACC = 57.32%
